In [1]:
import re
from typing import Dict, List
import pandas as pd


In [2]:
import sys
import os

# Get the project root directory (e.g., linkedin_jobs_analysis/)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add project root to sys.path
if project_root not in sys.path:
    sys.path.append(project_root)

# Now import from src
from config.analysis import ROLE_PATTERNS, SPECIAL_CASES
from src.analysis.analysis_utils import test_classifier, title_classifier


In [5]:
title_classifier('Engenheiro (a) de Software')

'Engenheiro de Software'

In [6]:
jobs_classified = pd.read_csv('jobs_classified.csv')
jobs_classified_tuple = list(zip(jobs_classified['job_title'], jobs_classified['classified_job_title']))

test_classifier(jobs_classified_tuple)

FAIL: 'Engenheiro(a) de Software (Fullstack) | IT Energy'
  Expected: Outros
  Got:      Engenheiro de Software
FAIL: 'Engenheiro(a) de Software Pleno – Plataforma de IA'
  Expected: Outros
  Got:      Engenheiro de Software
FAIL: 'Engenheiro(a) de Software Frontend'
  Expected: Outros
  Got:      Engenheiro de Software
FAIL: 'Engenheiro(a) de Software (.NET) Jr/Pleno | Electronic Trading'
  Expected: Outros
  Got:      Engenheiro de Software
FAIL: 'Engenheiro Computação/Software'
  Expected: Outros
  Got:      Engenheiro de Software
FAIL: 'Engenheiro(a) de Software (.NET) | IT Equities & Derivatives'
  Expected: Outros
  Got:      Engenheiro de Software
FAIL: 'Engenheiro(a) de Software IT Compliance Offshore'
  Expected: Outros
  Got:      Engenheiro de Software
FAIL: 'Engenheiro(a) de Software (.NET/AWS) | IT Payments & Corporate'
  Expected: Outros
  Got:      Engenheiro de Software
FAIL: 'Engenheiro(a) de Software Backend | FTS'
  Expected: Outros
  Got:      Engenheiro de Software

In [27]:
ROLE_PATTERNS['Cientista de Dados']

['\\b(data scientist|cientista de dados)\\b',
 '\\b(ml|machine learning|deep learning|llm|nlp)\\b',
 '(pesquisador|research).*(dados|data)',
 'ci[êe]ncia de dados',
 '\\bgenai\\b',
 '(computer vision|visão computacional)']